<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA_tfhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Dec 29 20:02:55 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# nlp aug
!pip install transformers nlpaug

# bert uncased base
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbu
!mkdir bert_base_uncased
!tar -xzf bbu -C bert_base_uncased

# bert cased base
!wget https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbc
!mkdir bert_base_cased
!tar -xzf bbc -C bert_base_cased

# albert base
!wget https://tfhub.dev/google/albert_base/1?tf-hub-format=compressed -O ab
!mkdir albert_base
!tar -xzf ab -C albert_base

# bert uncased large
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1?tf-hub-format=compressed -O blu
!mkdir bert_large_uncased
!tar -xzf blu -C bert_large_uncased



In [8]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
%tensorflow_version 2.x
import sys
sys.path.insert(1, '/content/drive/My Drive/GoogleQA')
import numpy as np
import pandas as pd
import os, gc
import codecs
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold, RepeatedKFold
from scipy.stats import spearmanr
from tqdm import tqdm
import tensorflow_hub as hub
# from albert_tokenization import FullTokenizer
import bert_tokenizer
from warmup_v2 import AdamWarmup, calc_train_steps
tf.get_logger().setLevel('ERROR') 
# from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps, get_custom_objects
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials
from functools import partial
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
import transformers
from nlpaug.util import Action
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0-rc1


##Load dataset, data augmentation

In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
aux_path = 'drive/My Drive/GoogleQA/Web Scrap/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
target_col = train.columns.tolist()[11:42]
# qa_aux = pd.read_csv(aux_path+'df_qa_extracted.csv')[['qa_id', 'question_viewed_times', 'question_votes', 'answer_counts', 'answer_votes']]
# quser_aux = pd.read_csv(aux_path+'df_question_user_extracted.csv')
# auser_aux = pd.read_csv(aux_path+'df_answer_user_extracted.csv')

# class MyLabelEncoder(object):
#     """safely handle unknown label"""
#     def __init__(self):
#         self.mapper = {}
#     def fit(self, X):
#         uniq_X = np.unique(X)
#         # reserve 0 for unknown
#         self.mapper = dict(zip(uniq_X, range(1, len(uniq_X) + 1)))
#         return self
#     def fit_transform(self, X):
#         self.fit(X)
#         return self.transform(X)
#     def _map(self, x):
#         return self.mapper.get(x, 0)
#     def transform(self, X):
#         return list(map(self._map, X))

# category_encoder = MyLabelEncoder().fit(train['category'])
# train['category'] = category_encoder.transform(train['category'])
# test['category'] = category_encoder.transform(test['category'])
# host_encoder = MyLabelEncoder().fit(train['host'])
# train['host'] = category_encoder.transform(train['host'])
# test['host'] = category_encoder.transform(test['host'])

aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute", top_k=20, aug_p=0.2)
def row_aug(data_df):
    q_id, a_id = [], []
    for col in target_col:
        a = data_df[col].value_counts(normalize=True)
        b = a[a<0.01].index.tolist()
        c = data_df.loc[data_df[col].isin(b), 'qa_id'].tolist()
        if 'question' in col:
            q_id.extend(c)
        else:
            a_id.extend(c)
    return list(set(q_id)), list(set(a_id))

def data_aug(data_df):
    q_id, a_id = row_aug(data_df) 
    for row in tqdm(q_id):    
        qt = data_df.loc[data_df.qa_id==row, 'question_title'].to_string(index=False)
        qb = data_df.loc[data_df.qa_id==row, 'question_body'].to_string(index=False)
        # for _ in range(1):
        tmp = data_df[data_df.qa_id==row].copy()
        tmp['question_title'] = aug.augment(qt)
        tmp['question_body'] = aug.augment(qb)
        data_df = pd.concat([data_df, tmp], ignore_index=True)

    for row in tqdm(a_id):    
        an = data_df.loc[data_df.qa_id==row, 'answer'].to_string(index=False)
        # for _ in range(1):
        tmp = data_df[data_df.qa_id==row].copy()
        tmp['answer'] = aug.augment(an)
        data_df = pd.concat([data_df, tmp], ignore_index=True)
    return data_df

def add_host_category(data_df):
    data_df['host'] = data_df.host.apply(lambda x: x.split('.')[0])
    data_df['question_title'] = data_df['category']+' '+data_df['host']+' '+data_df['question_title']
    return data_df

In [0]:
MAXLEN = 512 #@param {type:"slider", min:128, max:512, step:32}
BATCH_SIZE = 4 #@param {type:'slider', min:1, max:32, step:1}
NUM_EPOCHS = 3
NUM_CLASSES = 30
NUM_AUX = 6
LR = 5e-5
MIN_LR = 0
model_path = 'bert_base_uncased' #@param ['bert_base_uncased', 'bert_base_cased', 'bert_large_uncased', 'albert_base']
save_path = 'bbu6'
save_model = 'bbu6' 
CASED = False #@param ['True', 'False'] {type:"raw"}


## Bert tokenizer

In [0]:
def _trim_input(title, question, answer, max_sequence_length=MAXLEN,
                t_max_len=18, q_max_len=245, a_max_len=246):
    # t = tokenizer.tokenize(title)
    # q = tokenizer.tokenize(question)
    # a = tokenizer.tokenize(answer)
    t_len = len(title)
    q_len = len(question)
    a_len = len(answer)
    if (t_len + q_len + a_len +3) > max_sequence_length:
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + np.floor((t_max_len - t_len) / 2)
            q_max_len = q_max_len + np.ceil((t_max_len - t_len) / 2)
        else:
            t_new_len = t_max_len
        if a_max_len > a_len:
            a_new_len = a_len
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
        if t_new_len + a_new_len + q_new_len +3 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"
                    % (max_sequence_length, (t_new_len + a_new_len + q_new_len +3)))
        head_t_new_len = int(0.3 * t_new_len)
        tail_t_new_len = int(t_new_len - head_t_new_len)

        head_q_new_len = int(0.3 * q_new_len)
        tail_q_new_len = int(q_new_len - head_q_new_len)

        head_a_new_len = int(0.3 * a_new_len)
        tail_a_new_len = int(a_new_len - head_a_new_len)

        title = title[:head_t_new_len] + title[-tail_t_new_len:]
        question = question[:head_q_new_len] + question[-tail_q_new_len:]
        answer = answer[:head_a_new_len] + answer[-tail_a_new_len:]

    return title, question, answer

# token_dict = {}
# with codecs.open(os.path.join(model_path, 'assets/vocab.txt'), 'r', 'utf8') as reader:
#     for line in reader:
#         token = line.strip()
#         token_dict[token] = len(token_dict)
# tokenizer = Tokenizer(token_dict, cased=CASED)
tokenizer = bert_tokenizer.FullTokenizer(os.path.join(model_path, 'assets/vocab.txt'))


## Experimental engineered features

In [0]:
def aux_features(data_df):
    data_df["n_question_body"] = data_df["question_body"].apply(lambda x: len(x.strip())) / 10000
    data_df["n_question_title"] = data_df["question_title"].apply(lambda x: len(x.strip())) / 100
    data_df["n_answer"] = data_df["answer"].apply(lambda x: len(x.strip())) / 10000
    data_df["n_word_question_body"] = data_df["question_body"].apply(lambda x: len(x.split())) / 100
    data_df["n_word_question_title"] = data_df["question_title"].apply(lambda x: len(x.split())) / 100
    data_df["n_word_answer"] = data_df["answer"].apply(lambda x: len(x.split())) / 100
    data_df["is_question_text_in_title"] = data_df["question_title"].apply(lambda x: float(
        x.split()[0] in ["Do", "Can", "What", "How", "Is", "Which", "When", "Where", "can", "what", "how", "is",
                            "which", "What's", "does", "Does", "Are", "are", "Should", "Will"] or x[-1] in ["?"]))
    data_df["same_auther"] = (data_df["question_user_name"] == data_df["answer_user_name"]).astype(float)
    data_df["question_title_selection"] = (data_df["question_title"].apply(lambda x: "or" in x)).astype(float)

    for i in tqdm(range(len(data_df))):
        indirect = 0
        question_count = 0
        reason_explanation_words = 0
        choice_words = 0
        doc = nlp(data_df.loc[i, 'question_body'])
        for sent in doc.sents:
            if '?' in sent.text and '?' == sent.text[-1]:
                question_count += 1  # -> question_multi_intent
            for token in sent:
                if token.text.lower() == 'why':  # question_type_reason_explanation e.g index->102
                    reason_explanation_words += 1
                elif token.text.lower() == 'or':
                    choice_words += 1  # question_type_choice
        if question_count == 0:
            indirect = 1
        data_df.loc[i, 'indirect'] = indirect
        data_df.loc[i, 'question_count'] = question_count
        data_df.loc[i, 'reason_explanation_words'] = reason_explanation_words
        data_df.loc[i, 'choice_words'] = choice_words
  
    data_df['host'] = data_df.host.apply(lambda x: x.split('.')[0])
    data_df['question_title'] = data_df['category']+' '+data_df['host']+' '+data_df['question_title']

    return data_df

train, test = aux_features(train), aux_features(test)


## Convert data to inputs

In [0]:
def convert_data(data_df, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    global target_col
    ids, segments = [], []
    # q_title, q_body, answer = [], [], []
    for i in tqdm(range(len(data_df))):
        a = tokenizer.tokenize(data_df.loc[i, 'question_title'])
        b = tokenizer.tokenize(data_df.loc[i, 'question_body'])
        c = tokenizer.tokenize(data_df.loc[i, 'answer'])
        a, b, c = _trim_input(a, b, c)
        question = tokenizer.convert_tokens_to_ids(['[CLS]']+a+b+['[SEP]'])
        answer = tokenizer.convert_tokens_to_ids(c+['[SEP]'])
        ids.append(question + answer)
        segments.append([0] * len(question) + [1] * len(answer))
    # aux = data_df[['question_viewed_times', 'question_votes', 'answer_counts', 'answer_votes', 'answer_user_reputation', 'question_user_reputation']]
    # aux = aux.apply(lambda x: (x - x.mean()) / x.std())    
    # aux = data_df[['category', 'host']]    
    if branch == 'training':
        targets = data_df[target_col]
        # return [ids, segments], np.array(aux, dtype='int32'), np.array(targets)
        return [ids, segments], np.array(targets)
    else:
        # return [ids, segments], np.array(aux, dtype='int32')
        return [ids, segments]

## Data Generator

In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    # ML = MAXLEN
    out = np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x[:ML] for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)
    
def get_masks(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    # ML = MAXLEN
    out = np.array([np.concatenate([[1]*len(x), [padding] * (ML - len(x))]) if len(x) < ML else [1]*ML for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                X2 = get_masks([x[0] for x in d])    
                X3 = seq_padding([x[1] for x in d])
                # X3 = np.zeros_like(X1)

                # aux = np.array([x[2][0] for x in d]).reshape(-1, 1)
                # aux = tf.convert_to_tensor(aux, dtype=tf.int32)
                # aux2 = np.array([x[2][1] for x in d]).reshape(-1, 1)
                # aux2 = tf.convert_to_tensor(aux2, dtype=tf.int32)
                if self.branch == 'test':
                    yield [X1, X2, X3]
                    # yield [X1, X2, X3, aux, aux2]
                else:
                    Y = tf.convert_to_tensor([x[2] for x in d], dtype=tf.float32)
                    # Y2 = tf.convert_to_tensor([x[3] for x in d], dtype=tf.float32)
                    yield [X1, X2, X3], Y
                    # yield [X1, X2, X3, aux, aux2], Y


## Build model structure

In [0]:
def model_build(len_train, lr=LR, epochs=NUM_EPOCHS, bert_trainable=True):
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global MIN_LR
    # global LR
    global MAXLEN
    global model_path
    global NUM_AUX
 
    q_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_word_ids")
    q2_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_masks")
    q3_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_segment_ids")
    # input_category = keras.layers.Input((1,), dtype=tf.int32, name='input_category')
    # input_host = tf.keras.layers.Input((1,), dtype=tf.int32, name='input_host')

    bert_layer = hub.KerasLayer(model_path, trainable=bert_trainable)
    _, q_inputs  = bert_layer([q_in, q2_in, q3_in])
    q_outputs1 = keras.layers.GlobalAveragePooling1D()(q_inputs)
    q_outputs1 = keras.layers.Dropout(0.1)(q_outputs1)
    # q_outputs2 = keras.layers.GlobalMaxPooling1D()(q_inputs)
    # q_outputs2 = keras.layers.Dropout(0.1)(q_outputs2)

    # category_emb = keras.layers.Embedding(input_dim=len(category_encoder.mapper)+1, output_dim=32)(input_category)
    # category_emb = keras.layers.SpatialDropout1D(0.1)(category_emb)

    # host_emb = keras.layers.Embedding(input_dim=len(host_encoder.mapper)+1, output_dim=32)(input_host)
    # host_emb = keras.layers.SpatialDropout1D(0.1)(host_emb)

    # features_dense = keras.layers.concatenate([category_emb, host_emb], axis=1)
    # features_dense = keras.layers.Flatten()(features_dense)

    # dense = keras.layers.concatenate([q_outputs1, q_outputs2])
    outputs = keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(q_outputs1)
    # aoutputs = keras.layers.Dense(9, activation='sigmoid')(dense)

    # aux_out = keras.layers.Dense(NUM_AUX)(dense)
    model = keras.Model([q_in, q2_in, q3_in], outputs)

    decay_steps, warmup_steps = calc_train_steps(
        len_train,
        batch_size=BATCH_SIZE,
        epochs=epochs,
    )

    model.compile(
        loss=['binary_crossentropy'],
        # loss_weights = [2/3, 1/3],
        optimizer=AdamWarmup(
            decay_steps=decay_steps,
            warmup_steps=warmup_steps,
            lr=lr,
            min_lr=MIN_LR,
            ))
    #      optimizer=keras.optimizers.Adam(LR),
    # )

    return model

## train model

In [0]:
# pred = np.zeros((len(test), NUM_CLASSES))
# kf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1627)
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=0)
idx = [x for x in kf.split(train, groups=train.question_body)]

def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    return np.mean(rhos)

class IntervalEval(keras.callbacks.Callback):
    def __init__(self, stage=1, rho=-1):
        global NUM_EPOCHS
        global save_model
        super(keras.callbacks.Callback, self).__init__()
        self.stage = stage
        self.rho = rho
    def on_epoch_end(self, epoch, logs={}):
        val_pred = self.model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
        score = compute_spearmanr(val_pred, val_y)
        print('Spearman - {:.5f}'.format(score))
        # if self.stage==1:
        if score > self.rho:
            self.rho = score
            print('--Save Model--')
            self.model.save('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}-{:}.h5'.format(save_path, save_model, i, self.stage))


In [0]:
for i, (tr_idx, val_idx) in enumerate(idx[1:], 2):
    keras.backend.clear_session()
    gc.collect()

    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    tr = data_aug(tr)
    tr, val = add_host_category(tr), add_host_category(val)
    tr_x, tr_y = convert_data(tr)
    val_x, val_y = convert_data(val)

    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_y)))
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_y)), branch='valid')
    
    # stage 1 fine tunning
    ieval = IntervalEval()
    model = model_build(len(tr))
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )   

    # stage 2 fine tunning
    ieval = IntervalEval(stage=2)
    model = model_build(len(tr), lr=1e-4, epochs=4, bert_trainable=False)
    model.load_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}-1.h5'.format(save_path, save_model, i))
  
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=4,
        callbacks = [ieval]
    )

    gc.collect()

  0%|          | 1/345 [00:00<00:34,  9.95it/s]


Fold - 2



100%|██████████| 1216/1216 [00:06<00:00, 188.17it/s]


Train for 1348 steps
Epoch 1/3
  98/1348 [=>............................] - ETA: 14:38 - loss: 0.5445

In [0]:
# oof-prediction
model = model_build(len(tr), bert_trainable=False)
oof_pred = train[['qa_id']+target_col].copy()
oof_pred[target_col] = 0

for i, (tr_idx, val_idx) in enumerate(idx, 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]

    val_x, val_aux, val_y = convert_data(val)
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_aux, val_y)), branch='valid')

    model.load_weights('drive/My Drive/GoogleQA/Models/{}/{}-{}-2.h5'.format(save_path, save_model, i))
    val_pred = model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    oof_pred.loc[val_idx, target_col] += val_pred /2

    
    score = compute_spearmanr(val_y, val_pred)
    print('Spearman - {:.5f}'.format(score))

    gc.collect()

oof_pred.to_csv('drive/My Drive/GoogleQA/Data/train_oof_pred_bbu4.csv', index=False)

## oof-prediction hyperparameterize
1. Number of bins
2. Cutoffs

### Just use "90" transform

In [0]:
# bbu3: 0.42130
# bbu4: 0.43406
# bbu5: 0.42747
# lindada: 0.61590
oof_pred = pd.read_csv('drive/My Drive/GoogleQA/Data/train_oof_pred_bbu5.csv')
score = compute_spearmanr(np.floor(oof_pred[target_col].values*90)/90, train[target_col].values)
print('Total Score: {:.5f}'.format(score))

# just using "90" transformation
scores = []
for col, col_trues, col_pred in zip(target_col, train[target_col].values.T, (np.floor(oof_pred[target_col].values *90)/90).T):
    corr = spearmanr(col_trues, col_pred).correlation
    print('{} - {:.5f}'.format(col, corr))
    scores.append(corr)

hyper1 = pd.DataFrame(dict(col=target_col, score1=scores))

Total Score: 0.43406
question_asker_intent_understanding - 0.37733
question_body_critical - 0.63442
question_conversational - 0.45926
question_expect_short_answer - 0.30630
question_fact_seeking - 0.37559
question_has_commonly_accepted_answer - 0.43029
question_interestingness_others - 0.35603
question_interestingness_self - 0.50361
question_multi_intent - 0.59592
question_not_really_a_question - 0.13007
question_opinion_seeking - 0.48642
question_type_choice - 0.75009
question_type_compare - 0.43326
question_type_consequence - 0.25752
question_type_definition - 0.53540
question_type_entity - 0.51243
question_type_instructions - 0.78506
question_type_procedure - 0.35115
question_type_reason_explanation - 0.67556
question_type_spelling - 0.26873
question_well_written - 0.52203
answer_helpful - 0.23490
answer_level_of_information - 0.43260
answer_plausible - 0.15822
answer_relevance - 0.19437
answer_satisfaction - 0.33899
answer_type_instructions - 0.76599
answer_type_procedure - 0.27665

### Use lower (to 0) or upper (to 1) resetting

In [0]:
# bbu3: 0.44169
# bbu4: 0.45424
# bbu5: 0.44798 
# Lindada: 0.66098
corrparams = {
    'pct': hp.randint('pct', len(oof_pred)),
    'choice': hp.choice('choice', ['low', 'up'])
    }
def f_min(params, col):
    tmp = oof_pred.copy()[col].values
    if params['choice'] == 'low':
        loweridx = tmp.argsort()[:params['pct']]
        tmp[loweridx] = 0
    else:
        upperidx = tmp.argsort()[-params['pct']:]
        tmp[upperidx] = 1
    score = spearmanr(train[col].values, tmp).correlation
    return {'loss': -round(score, 5), 'status': STATUS_OK}

score, choice, pct = [], [], []
for col in target_col:
    f = partial(f_min, col=col)
    trials = Trials()
    best = fmin(f, corrparams, tpe.suggest, 100, rstate=np.random.RandomState(0), trials=trials, show_progressbar=False)
    print('{} - {} - {} - pct: {:.3f}'.format(
        col, 
        -trials.best_trial['result']['loss'],
        ['low', 'up'][best['choice']],
        best['pct']/len(oof_pred),
        # best['upper']/len(oof_pred)
        ))
    score.append(-trials.best_trial['result']['loss'])
    pct.append(round(best['pct']/len(oof_pred), 2))
    choice.append(['low', 'up'][best['choice']])

print(np.mean(score))

hyper2 = pd.DataFrame(dict(col = target_col, pct = pct, choice=choice, score2 = score))

question_asker_intent_understanding - 0.37807 - low - pct: 0.086
question_body_critical - 0.63432 - low - pct: 0.005
question_conversational - 0.52396 - low - pct: 0.881
question_expect_short_answer - 0.30577 - low - pct: 0.005
question_fact_seeking - 0.37506 - low - pct: 0.005
question_has_commonly_accepted_answer - 0.4881 - up - pct: 0.762
question_interestingness_others - 0.35939 - low - pct: 0.459
question_interestingness_self - 0.52263 - low - pct: 0.620
question_multi_intent - 0.61034 - low - pct: 0.536
question_not_really_a_question - 0.13796 - low - pct: 0.928
question_opinion_seeking - 0.48662 - low - pct: 0.005
question_type_choice - 0.76763 - low - pct: 0.459
question_type_compare - 0.54274 - low - pct: 0.953
question_type_consequence - 0.29275 - low - pct: 0.991
question_type_definition - 0.63611 - low - pct: 0.928
question_type_entity - 0.60212 - low - pct: 0.904
question_type_instructions - 0.79341 - up - pct: 0.393
question_type_procedure - 0.35181 - low - pct: 0.078
que

In [0]:
train['answer_type_procedure'].value_counts(normalize=True)

0.000000    0.704228
0.333333    0.205955
0.666667    0.053134
0.500000    0.020234
1.000000    0.016450
Name: answer_type_procedure, dtype: float64

### Use n-binning

In [0]:
# bbu3: 0.43042
# bbu4: 0.44542

def f1_min(params, col):
    tmp = oof_pred.copy()[col].values
    paramlist = [params['p'+str(i)] for i in range(len(params))]
    bins = np.percentile(tmp, paramlist)
    tmp = np.digitize(tmp, np.sort(bins)) / len(bins)
    score = spearmanr(train[col].values, tmp).correlation
    return {'loss': -round(score, 5), 'status': STATUS_OK}

def f2_min(params, col):
    cutoff = {'p'+str(i): hp.randint('p'+str(i), 101) for i in range(params['num_bin'])}
    f = partial(f1_min, col=col)
    trials = Trials()
    best = fmin(f, cutoff, tpe.suggest, 100, rstate=np.random.RandomState(0), trials=trials, 
                show_progressbar=False)
    # print('{} - {}'.format(col, -trials.best_trial['result']['loss']))
    score = -trials.best_trial['result']['loss']
    return {'loss': -round(score, 5), 'status': STATUS_OK, 'cutoff': list(best.values())}

score, cutoffs = [], []
for col in target_col:
    numbin = {'num_bin': hp.randint('num_bin', 19)+2}
    f = partial(f2_min, col=col)
    trials = Trials()
    best = fmin(f, numbin, tpe.suggest, 10, rstate=np.random.RandomState(0), trials=trials, show_progressbar=False)
    print('{} - {}'.format(col, -trials.best_trial['result']['loss']))
    score.append(-trials.best_trial['result']['loss'])
    cutoffs.append(trials.best_trial['result']['cutoff'])

print(np.mean(score))
hyper3 = pd.DataFrame(dict(col = target_col, cutoffs_pct = cutoffs, score3 = score))


question_asker_intent_understanding - 0.37851
question_body_critical - 0.63356
question_conversational - 0.5164
question_expect_short_answer - 0.30806
question_fact_seeking - 0.37662
question_has_commonly_accepted_answer - 0.48123
question_interestingness_others - 0.35821
question_interestingness_self - 0.51695
question_multi_intent - 0.60606
question_not_really_a_question - 0.13406
question_opinion_seeking - 0.48651
question_type_choice - 0.76248
question_type_compare - 0.53144
question_type_consequence - 0.27764
question_type_definition - 0.6253
question_type_entity - 0.59428
question_type_instructions - 0.79094
question_type_procedure - 0.35602
question_type_reason_explanation - 0.67527
question_type_spelling - 0.12838
question_well_written - 0.52302
answer_helpful - 0.24005
answer_level_of_information - 0.4319
answer_plausible - 0.16638
answer_relevance - 0.1986
answer_satisfaction - 0.34072
answer_type_instructions - 0.76742
answer_type_procedure - 0.28015
answer_type_reason_expla

In [0]:
# bbu 3: 0.44462
# bbu 4: 0.45486 (LB: 0.444)
# bbu 5: 0.44808 (LB: 0.441)
hyper = hyper1.merge(hyper2, on='col').merge(hyper3, on='col')
hyper.to_csv('drive/My Drive/GoogleQA/Data/hyper.csv', index=False)
hyper['score'] = hyper[['score1', 'score2', 'score3']].max(axis=1)
hyper['score'].mean()

0.45486470579748267

## test prediction

In [0]:
test_pred = pd.read_csv('drive/My Drive/GoogleQA/Data/test_oof_pred.csv')
hyper = pd.read_csv('drive/My Drive/GoogleQA/Data/hyper.csv')
pred = test_pred[target_col].copy().values

In [0]:
col1 = hyper.loc[hyper[['score1', 'score2', 'score3']].max(axis=1) == hyper.score1, 'col'].tolist()
col2 = hyper.loc[hyper[['score1', 'score2', 'score3']].max(axis=1) == hyper.score2, 'col'].tolist()
col3 = hyper.loc[hyper[['score1', 'score2', 'score3']].max(axis=1) == hyper.score3, 'col'].tolist()


for col in col1:
    colidx = target_col.index(col)
    pred[:, colidx] = (pred[:, colidx]//(1/90))/90

for col in col2:
    if hyper.loc[hyper.col==col, 'lower'].values == 1:
        pct = hyper.loc[hyper.col==col, 'lower'] - 0.01
    else:
        pct = hyper.loc[hyper.col==col, 'lower']
    changerow = int(len(test) * pct)
    colidx = target_col.index(col)
    rowidx = pred[:, colidx].argsort()[:changerow]
    pred[rowidx, colidx] = 0
    
for col in col3:
    cutoffs_pct = [int(x) for x in hyper.loc[hyper.col==col, 'cutoffs_pct'].strip('[]').split(',')]
    colidx = target_col.index(col)
    bins = np.percentile(pred[:, colidx], list(set(cutoffs_pct)))
    pred[:, colidx] = np.digitize(pred[:, colidx], np.sort(bins)) / len(bins)


['question_body_critical', 'answer_level_of_information']

## USE Model

In [0]:

# train['sentence'] = train['question'] + ' ' + train['answer']
embed1 = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
# embed2 = hub.load('https://tfhub.dev/google/nnlm-en-dim128/2')

def UniversalEmbedding(x):
    return embed1(tf.squeeze(tf.cast(x, tf.string)))

# def NNLMEmbedding(x):
#     return embed2(tf.squeeze(tf.cast(x, tf.string)))

def use_model():
    
    q = keras.layers.Input(shape=(1,), dtype=tf.string)
    qb = keras.layers.Input(shape=(1,), dtype=tf.string)
    a = keras.layers.Input(shape=(1,), dtype=tf.string)

    def hidden_layer(input_layer):
        x = keras.layers.Lambda(UniversalEmbedding, output_shape=(512, ))(input_layer)
        # embed2 = keras.layers.Lambda(NNLMEmbedding, output_shape=(128, ))(input_layer)
        # embed = keras.layers.concatenate([embed1, embed2])
        x = keras.layers.Dropout(0.2)(x)
        x = keras.backend.expand_dims(x, axis=1)
        # x = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(128, return_sequences=True))(x)
        # x = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(128, return_sequences=True))(x)
        # GP = keras.layers.GlobalMaxPooling1D()(x)
        # GP = keras.layers.Dropout(0.2)(GP)
        # AP = keras.layers.GlobalAveragePooling1D()(x)
        # AP = keras.layers.Dropout(0.2)(AP)
        # hidden = keras.layers.concatenate([GP, AP])
        # hidden = keras.layers.add([hidden, keras.layers.Dense(512, activation='relu')(hidden)])
        # hidden = keras.layers.add([hidden, keras.layers.Dense(512, activation='relu')(hidden)])

        x1 = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(256, return_sequences=True))(x)
        x2 = keras.layers.Bidirectional(keras.layers.CuDNNGRU(128, return_sequences=True))(x1)
        max_pool1 = keras.layers.GlobalMaxPooling1D()(x1)
        max_pool2 = keras.layers.GlobalMaxPooling1D()(x2)
        hidden = keras.layers.concatenate([max_pool1, max_pool2])
        return hidden

    q_embed, qb_embed, a_embed = hidden_layer(q), hidden_layer(qb), hidden_layer(a)
    all_embed = keras.layers.concatenate([q_embed, qb_embed, a_embed])
    dense = keras.layers.Dense(256, activation='relu')(all_embed)
    pred = keras.layers.Dense(30, activation='sigmoid')(dense)
    model = keras.Model(inputs=[q, qb, a], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=5e-3))
    # del embed
    # gc.collect()
    return model


In [0]:
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=0)
idx = [x for x in kf.split(train, groups=train.question_body)]
def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    return np.mean(rhos)

class IntervalEval(keras.callbacks.Callback):
    def __init__(self, val_data, label):
        super(keras.callbacks.Callback, self).__init__()
        self.val_data = val_data
        self.score = 0
        self.maxscore = 0
        self.label = label
        self.patience = 3
        self.count = 0
    def on_epoch_end(self, epoch, logs={}):
        # if epoch == 3:
        val_pred = self.model.predict(self.val_data, batch_size=16, verbose=1)
        score = compute_spearmanr(self.label, val_pred)
        print('Spearman - {:.5f}'.format(score))
        if self.maxscore>=0.3:
            self.patience=2
        if (score < self.score):
            self.score = score
            self.count+=1
            if self.count==self.patience:
                self.model.stop_training=True
        elif score > self.maxscore:
            self.score = score
            self.maxscore = score
            self.count = 0
            if self.maxscore > 0.3:
                self.model.save('drive/My Drive/GoogleQA/Models/{:}/use-model-{:}.h5'.format(save_model, i))
        else:
            self.score = score
            self.count = 0
          

In [0]:
with tf.Session() as session:
    keras.backend.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    for i, (tr_idx, val_idx) in enumerate(idx, 1):
        print('\nFold - {:}\n'.format(i))
        tr, val = train.loc[tr_idx], train.loc[val_idx]
        tr_x = [tr['question_title'], tr['question_body'], tr['answer']]
        val_x = [val['question_title'], val['question_body'], val['answer']]
        ieval = IntervalEval(val_data=val_x, label=val[target_col].values)
        model = use_model()
        history = model.fit(
            tr_x,
            tr[target_col],
            epochs=100, 
            batch_size=16,
            callbacks=[ieval])
        del model, tr, val, tr_x, val_x
        gc.collect()

# with tf.Session() as session:
#     keras.backend.set_session(session)
#     session.run(tf.global_variables_initializer())  
#     session.run(tf.tables_initializer())
#     model.load_weights('use-model-1.h5')
#     preds = model.predict(val['sentence'], batch_size=16, verbose=1)
#     score=compute_spearmanr(val[target_col].values, preds)
#     print('Spearman - {:.5f}'.format(score))


Fold - 1

Train on 4863 samples
Epoch 1/100
1216/1216 [==============================] - 33s 27ms/sample
Spearman - 0.35849
4863/4863 [==============================] - 170s 35ms/sample - loss: 0.3946
Epoch 2/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.36453
4863/4863 [==============================] - 85s 18ms/sample - loss: 0.3720
Epoch 3/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37807
4863/4863 [==============================] - 85s 18ms/sample - loss: 0.3628
Epoch 4/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37832
4863/4863 [==============================] - 86s 18ms/sample - loss: 0.3549
Epoch 5/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37996
4863/4863 [==============================] - 86s 18ms/sample - loss: 0.3462
Epoch 6/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37266
4863/4863 [=======================